In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from transformers import BertForSequenceClassification


In [2]:
# Install CUDA Toolkit
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-0_11.0.3-1_amd64.deb
!dpkg -i cuda-11-0_11.0.3-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda

# Install cuDNN (adjust version accordingly)
!wget https://developer.download.nvidia.com/compute/redist/cudnn/v8.0.5/cudnn-11.0-linux-x64-v8.0.5.39.tgz
!tar -xzvf cudnn-11.0-linux-x64-v8.0.5.39.tgz
!cp cuda/include/cudnn*.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

--2024-05-09 02:12:34--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-0_11.0.3-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2446 (2.4K) [application/x-deb]
Saving to: ‘cuda-11-0_11.0.3-1_amd64.deb.1’

cuda-11-0_11.0.3-1_ 100%[===================>]   2.39K  --.-KB/s    in 0s      

2024-05-09 02:12:34 (259 MB/s) - ‘cuda-11-0_11.0.3-1_amd64.deb.1’ saved [2446/2446]

(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack cuda-11-0_11.0.3-1_amd64.deb ...
Unpacking cuda-11-0 (11.0.3-1) over (11.0.3-1) ...
dpkg: dependency problems prevent configuration of cuda-11-0:
 cuda-11-0 depends on cuda-runtime-11-0 (>= 11.0.3); however:
  Package cuda-runtime-11-0 is not installed.
 cuda-11-0 depends on cuda

In [3]:
!nvidia-smi

Thu May  9 02:13:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import torch
torch.cuda.is_available()


True

In [5]:
!pip install transformers --upgrade




In [6]:
import chardet

In [7]:
train_datapath = f'train.csv'
#df = pd.read_csv(datapath,index_col=False)
#df.reset_index(drop=True, inplace=True)
with open('train.csv', 'rb') as f:
    encoding_train = chardet.detect(f.read())['encoding']

train_df = pd.read_csv(train_datapath, encoding=encoding_train)

val_datapath = f'validate.csv'
#df = pd.read_csv(datapath,index_col=False)
#df.reset_index(drop=True, inplace=True)
with open('validate.csv', 'rb') as f:
    encoding_val = chardet.detect(f.read())['encoding']
val_df = pd.read_csv(val_datapath, encoding=encoding_val)



test_datapath = f'test.csv'
#df = pd.read_csv(datapath,index_col=False)
#df.reset_index(drop=True, inplace=True)
with open('test.csv', 'rb') as f:
    encoding_test = chardet.detect(f.read())['encoding']
test_df = pd.read_csv(val_datapath, encoding=encoding_test)


In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
labels = {'0':0,
          '1':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label'].astype(str)]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)


    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [10]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()
    #cls_embeddings_with_text = []
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                #labels = [label for label in train_label]
                #decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

                #for text,embedding,label in zip(decoded_texts,output,labels):
                  #cls_embeddings_with_text.append((text,embedding,label))

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
    #return cls_embeddings_with_text

In [11]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0

    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)


              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc


    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))


NameError: name 'df' is not defined

In [26]:
EPOCHS = 20
num_classes = 2
model = BertClassifier()
LR = 2e-5

train(model, train_df, val_df, LR, EPOCHS)

100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Epochs: 1 | Train Loss:  0.051 | Train Accuracy:  0.525 | Val Loss:  0.048 | Val Accuracy:  0.750


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Epochs: 2 | Train Loss:  0.043 | Train Accuracy:  0.825 | Val Loss:  0.038 | Val Accuracy:  0.775


100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


...

Epochs: 18 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.023 | Val Accuracy:  0.875


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Epochs: 19 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.023 | Val Accuracy:  0.900


100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


Epochs: 20 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.023 | Val Accuracy:  0.875


In [27]:
evaluate(model, test_df)

Test Accuracy:  0.875


In [17]:
EPOCHS = 20
num_classes = 2
model = BertClassifier()
LR = 1e-6
#voir si ca plafonne
train(model, train_df, val_df, LR, EPOCHS)

100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


Epochs: 1 | Train Loss:  0.052 | Train Accuracy:  0.525 | Val Loss:  0.053 | Val Accuracy:  0.550


100%|██████████| 3/3 [00:02<00:00,  1.04it/s]


Epochs: 2 | Train Loss:  0.050 | Train Accuracy:  0.550 | Val Loss:  0.052 | Val Accuracy:  0.600


100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


...

Epochs: 18 | Train Loss:  0.044 | Train Accuracy:  0.750 | Val Loss:  0.049 | Val Accuracy:  0.675


100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


Epochs: 19 | Train Loss:  0.042 | Train Accuracy:  0.750 | Val Loss:  0.046 | Val Accuracy:  0.725


100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


Epochs: 20 | Train Loss:  0.043 | Train Accuracy:  0.750 | Val Loss:  0.045 | Val Accuracy:  0.775


In [18]:
evaluate(model, test_df)

Test Accuracy:  0.675


In [19]:
from transformers import AutoTokenizer, BertForSequenceClassification

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)


    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

In [20]:
EPOCHS = 20
num_classes = 2
model = BertClassifier()
LR = 1e-6

train(model, train_df, val_df, LR, EPOCHS)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epochs: 1 | Train Loss:  0.052 | Train Accuracy:  0.500 | Val Loss:  0.053 | Val Accuracy:  0.475


100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


Epochs: 2 | Train Loss:  0.051 | Train Accuracy:  0.500 | Val Loss:  0.053 | Val Accuracy:  0.475


100%|██████████| 3/3 [00:03<00:00,  1.00s/it]


...

Epochs: 18 | Train Loss:  0.033 | Train Accuracy:  1.000 | Val Loss:  0.045 | Val Accuracy:  0.800


100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epochs: 19 | Train Loss:  0.031 | Train Accuracy:  1.000 | Val Loss:  0.045 | Val Accuracy:  0.800


100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epochs: 20 | Train Loss:  0.030 | Train Accuracy:  1.000 | Val Loss:  0.044 | Val Accuracy:  0.800


In [21]:
evaluate(model, test_df)

Test Accuracy:  0.800


In [22]:
EPOCHS = 20
num_classes = 2
model = BertClassifier()
LR = 2e-5

train(model, train_df, val_df, LR, EPOCHS)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epochs: 1 | Train Loss:  0.062 | Train Accuracy:  0.475 | Val Loss:  0.050 | Val Accuracy:  0.500


100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epochs: 2 | Train Loss:  0.036 | Train Accuracy:  0.875 | Val Loss:  0.043 | Val Accuracy:  0.750


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


...

Epochs: 18 | Train Loss:  0.003 | Train Accuracy:  1.000 | Val Loss:  0.032 | Val Accuracy:  0.750


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Epochs: 19 | Train Loss:  0.003 | Train Accuracy:  1.000 | Val Loss:  0.032 | Val Accuracy:  0.750


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


Epochs: 20 | Train Loss:  0.003 | Train Accuracy:  1.000 | Val Loss:  0.032 | Val Accuracy:  0.775


In [23]:
evaluate(model, test_df)

Test Accuracy:  0.775


In [ ]:
torch.save(model.state_dict(), "bert_classifier.pth")

In [24]:
def predict_class(text, model, tokenizer,  max_length=128):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        _, preds = torch.max(outputs, dim=1)
        return "expert" if preds.item() == 1 else "beg"

In [28]:
predict_class('Tomatoes are delicious and versatile fruits that are enjoyed all around the world. They come in many different shapes, sizes, and colors, ranging from small cherry tomatoes to large beefsteak tomatoes. Some are round, while others are oval or even pear-shaped. Tomatoes can be red, yellow, orange, green, or even purple, adding a pop of color to any meal.', model,tokenizer)

tensor([[ 2.1153, -1.5393]], device='cuda:0')


'beg'

In [29]:
text1 = 'Ballet, a discipline merging athleticism, artistry, and cultural heritage, embodies a profound narrative that transcends time and space. This intricate art form traces its origins to the lavish courts of Renaissance Italy, where it flourished as a courtly dance, and has since evolved into a global phenomenon, captivating audiences with its grace, precision, and emotive storytelling. At its core, ballet is a harmonious fusion of movement, music, and narrative, expressed through the language of the body. Dancers, finely tuned instruments of expression, undergo rigorous training to master the technical demands of classical ballet technique, from the ethereal beauty of a perfectly executed arabesque to the explosive power of a grand allegro jump. Yet, beyond the physical prowess lies a profound emotional depth, as dancers imbue each movement with intention, nuance, and soul, inviting audiences into a world of heightened emotion and transcendence. Ballet repertoire spans centuries of human experience, drawing inspiration from mythology, literature, history, and contemporary life. From the timeless tragedy of "Swan Lake" to the romantic allure of "Giselle" and the avant-garde innovations of modern choreographers, ballet serves as a mirror reflecting the human condition in all its complexity, beauty, and vulnerability. Through movement, dancers convey universal themes of love, loss, longing, and triumph, evoking a visceral response that resonates with audiences across generations and cultures.Behind the scenes, ballet is a labor of love, requiring unwavering dedication, discipline, and sacrifice. Dancers devote countless hours to honing their craft, pushing the limits of their physical and artistic abilities, and enduring the relentless pursuit of perfection. From the ballet studio to the stage, they navigate a world of intense competition, auditioning for coveted roles, striving for artistic recognition, and grappling with the pressures of performance. Yet, for all its challenges, ballet offers moments of sublime beauty and transcendence that transcend the boundaries of language and culture. In the ephemeral space of the theater, dancers and audiences alike are transported to a realm where time stands still, where dreams take flight, and where the human spirit soars. In this shared experience of wonder and awe, ballet becomes more than mere entertainment; it becomes a celebration of the human spirit, a testament to the power of art to inspire, uplift, and transform lives.'

In [32]:
predict_class(text1, model,tokenizer)

tensor([[-1.7478,  1.5912]], device='cuda:0')


'expert'

In [30]:
text2 = 'Ballet is a beautiful and graceful form of dance that tells stories through movement and music. It is like a fairy tale come to life on stage, where dancers float and twirl in elegant costumes, captivating audiences with their skill and artistry. In ballet, dancers use their bodies to express emotions and tell stories without words. They learn special steps and poses called positions and technique that help them move gracefully across the stage. These movements can be slow and gentle, like a swan gliding on a lake, or fast and lively, like a princess dancing at a royal ball. Ballet performances often have elaborate sets and costumes that transport the audience to magical worlds filled with princes and princesses, fairies and witches. The music, composed by talented musicians, adds to the enchantment of the performance, sweeping listeners away on a journey of imagination and wonder.Dancers start learning ballet at a young age, practicing in ballet studios with special floors and mirrors to help them perfect their technique. They work hard in classes, stretching and strengthening their bodies to become strong and flexible dancer. Ballet has many famous stories, like "Swan Lake," "The Nutcracker," and "Cinderella." These timeless tales have been passed down through generations and continue to inspire audiences around the world. Whether you are watching a ballet performance for the first time or practicing ballet in a dance class, there is something magical about the beauty and grace of this timeless art form. So put on your ballet shoes, take a deep breath, and let the music carry you away into a world of enchantment and dreams.'

In [31]:
predict_class(text2, model,tokenizer)

tensor([[-1.4837,  1.5586]], device='cuda:0')


'expert'